# CART 树回归

CART（Classification And Regression Trees，分类回归树）的树构建算法。该算法既可以用于分类还可以用于回归，接着，利用CART算法构建 回归树并介绍其中的树剪枝技术（该技术的主要目的是防止树的过拟合）。之后引入了一个更高 级的模型树算法。与回归树的做法（在每个叶节点上使用各自的均值做预测）不同，该算法需要 在每个叶节点上都构建出一个线性模型。在这些树的构建算法中有一些需要调整的参数。

决策树不断将数据切分成小数据集，直到所有目标变量完全相同，或者数据不能再切分为止。决策树是一种贪心算法，它要在给定时间内做出最佳选择，但并不关心能否达到全局最优。

优点：可以对复杂和非线性的数据建模。

缺点：结果不易理解。

适用数据类型：数值型和标称型数据

第3章使用的树构建算法是ID3。ID3的做法是每次选取当前最佳的特征来分割数据，并按照 该特征的所有可能取值来切分。也就是说，如果一个特征有4种取值，那么数据将被切成4份。一 旦按某特征切分后，该特征在之后的算法执行过程中将不会再起作用，所以有观点认为这种切分 方式过于迅速。另外一种方法是二元切分法，即每次把数据集切成两份。如果数据的某特征值等 于切分所要求的值，那么这些数据就进入树的左子树，反之则进入树的右子树。

除了切分过于迅速外，ID3算法还存在另一个问题，它不能直接处理连续型特征。只有事先将连续型特征转换成离散型，才能在ID3算法中使用。但这种转换过程会破坏连续型变量的内在性质。而使用二元切分法则易于对树构建过程进行调整以处理连续型特征。具体的处理方法是： 如果特征值大于给定值就走左子树，否则就走右子树。另外，二元切分法也节省了树的构建时间， 但这点意义也不是特别大，因为这些树构建一般是离线完成，时间并非需要重点关注的因素。

CART是十分著名且广泛记载的树构建算法，它使用二元切分来处理连续型变量。对CART 稍作修改就可以处理回归问题。第3章中使用香农熵来度量集合的无组织程度。如果选用其他方 法来代替香农熵，就可以使用树构建算法来完成回归。

下面将实观CART算法和回归树。回归树与分类树的思路类似，但叶节点的数据类型不是离 散型，而是连续型。

## 连续和离散型特征的树的构建

在树的构建过程中，需要解决多种类型数据的存储问题。与第3章类似，这里将使用一部字典来存储树的数据结构，该字典将包含以下4个元素。  待切分的特征。待切分的特征值。右子树。当不再需要切分的时候，也可以是单个值。左子树。与右子树类似。

第3章用一部字典来存储每个切分，但该字典可以包含两个或两个以上的值。而CART算法只做二元切分，所以这里可以固定树的数据结构。树包含左键和右键，可以存储另一棵子树或者单个值。字典还包含特征和特征值这两个键，它们给出切分算法 所有的特征和特征值。。Python 具有足够的灵活性，可以直接使用字典来存储树结构而无须另外自定义一个类，从而有效地减少 代码量。Python不是一种强类型编程语言 ，因此接下来会看到，树的每个分枝还可以再包含其他 树、数值型数据甚至是向量。

本章将构建两种树：第一种是9.4节的回归树（regression tree），其每个叶节点包含单个值； 第二种是9.5节的模型树（model tree），其每个叶节点包含一个线性方程。

函数createTree()的伪代码大致如下：

    找到最佳的待切分特征：
        如果该节点不能再分，将该节点存为叶节点 
        执行二元切分 
        在右子树调用createTree()方法 
        在左子树调用createTree()方法

In [4]:
import regTrees
from numpy import *
test = mat(eye(4))
mat0, mat1 = regTrees.binSplitDataSet(test, 1, 0.5)
print(mat0)
print(mat1)

[[0. 1. 0. 0.]]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


那么如何实现数据的 切分呢？怎么才能知道是否已经充分切分呢？这些问题的答案取决于叶节点的建模方式。回归树 假设叶节点是常数值，这种策略认为数据中的复杂关系可以用树结构来概括。

为成功构建以分段常数为叶节点的树，需要度量出数据的一致性。第3章使用树进行分类，**会在给定节点时计算数据的混乱度**。 那么如何计算连续型数值的混乱度呢？事实上， 在数据集上计算混乱度是非常简单的。 首先计算所有数据的均值， **然后计算每条数据的值到均值的差值**。 为了对正负差值同等看待， 一般使用绝对值或平方值来代替上述差值。 上述做法有点 类似于前面介绍过的统计学中常用的方差计算。唯一的不同就是，方差是平方误差的均值（均方差）， 而这里需要的是平方误差的总值（总方差）。 总方差可以通过均方差乘以数据集中样本点的个数来得到。

### 构建树

函数chooseBestSplit()最复杂，该函数的目标是找到数据集切分的最佳位置。它遍历所有的特征及其可能的取值来找到使误差最小化的切分阈值。该函数的伪代码大致如下：

    对每个特征：
        对每个特征值： 
            将数据集切分成两份 
            计算切分的误差 
            如果当前误差小于当前最小误差，那么将当前切分设定为最佳切分并更新最小误差
    返回最佳切分的特征和阈值


In [5]:
myData = regTrees.loadDataSet('ex00.txt')
myMat = mat(myData)
regTrees.createTree(myMat)

{'spInd': 0,
 'spVal': 0.48813,
 'left': 1.0180967672413792,
 'right': -0.04465028571428572}

In [6]:
#另一个数据集
myData1 = regTrees.loadDataSet('ex0.txt')
myMat1 = mat(myData1)
regTrees.createTree(myMat1)

{'spInd': 1,
 'spVal': 0.39435,
 'left': {'spInd': 1,
  'spVal': 0.582002,
  'left': {'spInd': 1,
   'spVal': 0.797583,
   'left': 3.9871632,
   'right': 2.9836209534883724},
  'right': 1.980035071428571},
 'right': {'spInd': 1,
  'spVal': 0.197834,
  'left': 1.0289583666666666,
  'right': -0.023838155555555553}}

可以检查一下该树的结构以确保树中包含5个叶节点。读者也可以在更复杂的数据集上构建 回归树并观察实验结果。

到现在为止，已经完成回归树的构建，但是需要某种措施来检查构建过程否得当。

## 树剪枝

一棵树如果节点过多，表明该模型可能对数据进行了“过拟合”。那么，如何判断是否发生了 过拟合？前面章节中使用了测试集上某种交叉验证技术来发现过拟合，决策树亦是如此。本节将 对此进行讨论，并分析如何避免过拟合。

通过降低决策树的复杂度来避免过拟合的过程称为剪枝（pruning）。其实本章前面已经进行 过剪枝处理。在函数chooseBestSplit()中的提前终止条件，实际上是在进行一种所谓的预剪 枝（prepruning）操作。另一种形式的剪枝需要使用测试集和训练集，称作后剪枝（postpruning）。 本节将分析后剪枝的有效性，但首先来看一下预剪枝的不足之处。

### 预剪枝

上节两个简单实验的结果还是令人满意的，但背后存在一些问题。树构建算法其实对输入的 参数tolS和tolN非常敏感，如果使用其他值将不太容易达到这么好的效果。为了说明这一点， 在Python提示符下输入如下命令：

In [9]:
#regTrees.createTree(myMat, ops=(0,1))

与上节中只包含两个节点的树相比，这里构建的树过于臃肿，它为数据集中每个样本都分配了一个叶节点。

停止条件tolS对误差的数量级十分敏感。如果在选项 中花费时间并对上述误差容忍度取平方值，或许也能得到仅有两个叶节点组成的树：然而，通过不断修改停止条件来得到合理结果并不是很好的办法。事实上，我们常常甚至不确 定到底需要寻找什么样的结果。这正是机器学习所关注的内容，计算机应该可以给出总体的概貌。

下节将讨论后剪枝，即利用测试集来对树进行剪枝。由于不需要用户指定参数，后剪枝是一 个更理想化的剪枝方法。

### 后剪枝

使用后剪枝方法需要将数据集分成测试集和训练集。首先指定参数，使得构建出的树足够大、足够复杂，便于剪枝。接下来从上而下找到叶节点，用测试集来判断将这些叶节点合并是否能降 低测试误差。如果是的话就合并。函数prune()的伪代码如下：

    基于已有的树切分测试数据：
        如果存在任一子集是一棵树，则在该子集递归剪枝过程 
        计算将当前两个叶节点合并后的误差 
        计算不合并的误差 
        如果合并会降低误差的话，就将叶节点合并

In [ ]:
myData2 = regTrees.loadDataSet('ex2.txt')
myMat2 = mat(myData2)
myTree = regTrees.createTree(myMat2, ops = (0, 1))
#导入测试数据
myDataTest = regTrees.loadDataSet('ex2test.txt')
myMat2Test = mat(myDataTest)
regTrees.prune(myTree, myMat2Test)

可以看到，大量的节点已经被剪枝掉了，但没有像预期的那样剪枝成两部分，这说明后剪枝 可能不如预剪枝有效。一般地，为了寻求最佳模型可以同时使用两种剪枝技术。

下节将重用部分已有的树构建代码来创建一种新的树。该树仍采用二元切分，但叶节点不再 是简单的数值，取而代之的是一些线性模型。

## 模型树

用树来对数据建模，除了把叶节点简单地设定为常数值之外，还有一种方法是把叶节点设定 为分段线性函数，这里所谓的分段线性（piecewise linear）是指模型由多个线性片段组成。如果 读者仍不清楚，下面很快就会给出样例来帮助理解。考虑图9-4中的数据，如果使用两条直线拟合是否比使用一组常数来建模好呢 ？答案显而易见。可以设计两条分别从0.0～0.3、从0.3～1.0 的直线，于是就可以得到两个线性模型。因为数据集里的一部分数据（0.0～0.3）以某个线性模型建模，而另一部分数据（0.3～1.0）则以另一个线性模型建模，因此我们说采用了所谓的分段 线性模型。(两个线性模型的组合)

决策树相比于其他机器学习算法的优势之一在于结果更易理解。很显然，两条直线比很多节 点组成一棵大树更容易解释。模型树的可解释性是它优于回归树的特点之一。另外，模型树也具 有更高的预测准确度。前面的代码稍加修改就可以在叶节点生成线性模型而不是常数值。下面将利用树生成算法对 数据进行切分，且每份切分数据都能很容易被线性模型所表示。该算法的关键在于误差的计算。

前面已经给出了树构建的代码，但是这里仍然需要给出每次切分时用于误差计算的代码。不知道读者是否还记得之前createTree()函数里有两个参数从未改变过。回归树把这两个参数固定，而此处略做修改，从而将前面的代码重用于模型树。

下一个问题就是，为了找到最佳切分，应该怎样计算误差呢？前面用于回归树的误差计算方 法这里不能再用。稍加变化，对于给定的数据集，应该先用线性的模型来对它进行拟合，然后计 算真实的目标值与模型预测值间的差值。最后将这些差值的平方求和就得到了所需的误差。

In [13]:
myMat2 = mat(regTrees.loadDataSet('ex2.txt'))
regTrees.createTree(myMat2, regTrees.modelLeaf, regTrees.modelErr, (1,10))

{'spInd': 0,
 'spVal': 0.499171,
 'left': {'spInd': 0,
  'spVal': 0.729397,
  'left': {'spInd': 0,
   'spVal': 0.790312,
   'left': {'spInd': 0,
    'spVal': 0.841547,
    'left': {'spInd': 0, 'spVal': 0.910975, 'left': matrix([[-85.38134691],
             [195.2916547 ]]), 'right': matrix([[-198.60707966],
             [ 335.8875464 ]])},
    'right': matrix([[-413.78611049],
            [ 618.92243507]])},
   'right': matrix([[-358.95313076],
           [ 589.03437394]])},
  'right': {'spInd': 0,
   'spVal': 0.613004,
   'left': {'spInd': 0, 'spVal': 0.666452, 'left': matrix([[-37.38215781],
            [209.4219232 ]]), 'right': matrix([[-259.91915006],
            [ 565.77696463]])},
   'right': {'spInd': 0, 'spVal': 0.549814, 'left': matrix([[-212.66006427],
            [ 557.78748581]]), 'right': matrix([[ 28.11573016],
            [146.92675371]])}}},
 'right': {'spInd': 0,
  'spVal': 0.382037,
  'left': {'spInd': 0, 'spVal': 0.446196, 'left': matrix([[-314.69568607],
          

可以看到， 该代码以0.285 477为界创建了两个模型， 而图9-4的数据实际在0.3处分段。 createTree()生成的这两个线性模型分别是y=3.468+1.1852和y=0.001 6985+11.964 77x，与用于生成该数据的真实模型非常接近。该数据实际是由模型y=3.5+1.0x和y=0+12x再加上高 斯噪声生成的。

模型树、回归树以及第8章里的其他模型，哪一种模型更好呢？一个比较客观的方法是计算相关系数， 也称为R2值。 该相关系数可以通过调用NumPy库中的命令 corrcoef(yHat, y, rowvar=0)来求解，其中yHat是预测值，y是目标变量的实际值。(回归问题的衡量指标)

前一章使用了标准的线性回归法，本章则使用了树回归法，下面将通过实例对二者进行比较， 最后用函数corrcoef()来分析哪个模型是最优的。

### 树回归与标准回归的比较

该数据是我从多个骑自行车的人那里收集得到的。图中给出 骑自行车的速度和人的智商之间的关系。下面将基于该数据集建立多个模型并在另一个测试集上 进行测试。对应的训练集数据保存在文件bikeSpeedVsIq_train.txt中，而测试集数据保存在文件 bikeSpeedVsIq_test.txt中。$R^2$ 值越接近1.0越好

In [17]:
#利用该数据创建一棵回归树
trainMat = mat(regTrees.loadDataSet('bikeSpeedVsIq_train.txt'))
testMat = mat(regTrees.loadDataSet('bikeSpeedVsIq_test.txt'))
myTree = regTrees.createTree(trainMat, ops = (1,20))
yHat = regTrees.createForeCast(myTree, testMat[:,0])
corrcoef(yHat, testMat[:,1], rowvar= 0)[0,1]
       

0.9640852318222141

In [22]:
#创建一棵模型树：
myTree = regTrees.createTree(trainMat, regTrees.modelLeaf, regTrees.modelErr, (1,20))
yHat = regTrees.createForeCast(myTree, testMat[:,0], regTrees.modelTreeEval)
corrcoef(yHat, testMat[:,1], rowvar= 0)[0,1]

0.9760412191380593

$R^2$ 值越接近1.0越好，所以从上面的结果可以看出，这里模型树的结果比回归树 好。下面再看看标准的线性回归效果如何，这里无须导入第8章的任何代码，本章已实现过一个 线性方程求解函数

In [23]:
ws, X, Y = regTrees.linearSolve(trainMat)
ws

matrix([[37.58916794],
        [ 6.18978355]])

In [25]:
for i in range(shape(testMat)[0]):
  yHat[i] = testMat[i,0] * ws[1,0] + ws[0,0]

In [26]:
corrcoef(yHat, testMat[:,1], rowvar= 0)[0,1]

0.9434684235674763

可以看到，该方法在R 2 值上的表现上不如上面两种树回归方法。所以，树回归方法在预测复 杂数据时会比简单的线性模型更有效，相信读者对这个结论也不会感到意外。下面将展示如何对 回归模型进行定性的比较。

## 小结

数据集中经常包含一些复杂的相互关系，使得输入数据和目标变量之间呈现非线性关系。对 这些复杂的关系建模，一种可行的方式是使用树来对预测值分段，包括分段常数或分段直线。一 般采用树结构来对这种数据建模。相应地，若叶节点使用的模型是分段常数则称为回归树，若叶 节点使用的模型是线性回归方程则称为模型树。

CART算法可以用于构建二元树并处理离散型或连续型数据的切分。若使用不同的误差准则， 就可以通过CART算法构建模型树和回归树。该算法构建出的树会倾向于对数据过拟合。一棵过 拟合的树常常十分复杂，剪枝技术的出现就是为了解决这个问题。两种剪枝方法分别是预剪枝（在 树的构建过程中就进行剪枝）和后剪枝（当树构建完毕再进行剪枝），预剪枝更有效但需要用户 定义一些参数。

Tkinter是Python的一个GUI工具包。虽然并不是唯一的包，但它最常用。利用Tkinter，我们 可以轻松绘制各种部件并灵活安排它们的位置。另外，可以为Tkinter构造一个特殊的部件来显示 Matplotlib绘出的图。所以，Matplotlib和Tkinter的集成可以构建出更强大的GUI，用户可以以更自 然的方式来探索机器学习算法的奥妙。

本章是回归的最后一章，希望读者没有错过。接下来我们将离开监督学习的岛屿，驶向无监 督学习的未知港湾。在回归和分类（监督学习）中，目标变量的值是已知的。在后面的章节将会 看到，无监督学习中上述条件将不再成立。下一章的主要内容是K-均值聚类算法。